In [1]:
import numpy as np
import pandas as pd

In [2]:
dataSet = pd.read_csv("/kaggle/input/imdb-dataset-1920-2023/FinalList.csv")
dataSet.head()

,Unnamed: 0,tconst,averageRating,numVotes,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,3,tt0000004,5.6,178,short,Un bon bock,Un bon bock,0,1892,12,"Animation,Short"
1,8,tt0000009,5.3,204,movie,Miss Jerry,Miss Jerry,0,1894,45,Romance
2,144,tt0000147,5.3,469,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,100,"Documentary,News,Sport"
3,198,tt0000218,5.9,662,short,L'affaire Dreyfus,L'affaire Dreyfus,0,1899,13,"Biography,Drama,Short"
4,252,tt0000335,6.1,55,short,Soldiers of the Cross,Soldiers of the Cross,0,1900,40,"Biography,Drama,Short"


In [3]:
dataSet.columns

Index(['Unnamed: 0', 'tconst', 'averageRating', 'numVotes', 'titleType',
       'primaryTitle', 'originalTitle', 'isAdult', 'startYear',
       'runtimeMinutes', 'genres'],
      dtype='object')

In [4]:
dataSet.isnull().sum()

Unnamed: 0        0
tconst            0
averageRating     0
numVotes          0
titleType         0
primaryTitle      0
originalTitle     0
isAdult           0
startYear         0
runtimeMinutes    0
genres            0
dtype: int64

In [5]:
dataSet = dataSet.drop(columns=["Unnamed: 0","isAdult","originalTitle"],axis=1)
dataSet["movieId"] = range(len(dataSet))
dataSet.head()

,tconst,averageRating,numVotes,titleType,primaryTitle,startYear,runtimeMinutes,genres,movieId
0,tt0000004,5.6,178,short,Un bon bock,1892,12,"Animation,Short",0
1,tt0000009,5.3,204,movie,Miss Jerry,1894,45,Romance,1
2,tt0000147,5.3,469,movie,The Corbett-Fitzsimmons Fight,1897,100,"Documentary,News,Sport",2
3,tt0000218,5.9,662,short,L'affaire Dreyfus,1899,13,"Biography,Drama,Short",3
4,tt0000335,6.1,55,short,Soldiers of the Cross,1900,40,"Biography,Drama,Short",4


In [6]:
dataSet["genres"].describe()

count     862042
unique      1919
top        Drama
freq       71960
Name: genres, dtype: object

In [7]:
userMovies_rate = {dataSet.iloc[72505]["movieId"]:8.0 ,dataSet.iloc[507882]["movieId"]:6.0 ,dataSet.iloc[198210]["movieId"]:7.2}
for key in userMovies_rate.keys():
    primaryTitle = dataSet[dataSet["movieId"] == key]["primaryTitle"].values
    print(primaryTitle)

['Friends']
['Avatar: The Way of Water']
['The Tree of Life']


In [8]:
userProfile = pd.DataFrame()

for userMovie_id,userMovie_rate in userMovies_rate.items():
    rating = userMovie_rate
    print("Movie id: ", userMovie_id, " rating: ", rating)
    genres = (dataSet[dataSet["movieId"] == userMovie_id]["genres"].str.split(",")).iloc[0]
    data = {}
    for genre in genres:
        print(genre)
        if genre not in userProfile.columns:
            userProfile.loc[1,genre]= rating
        else:
            print(rating)
            userProfile.loc[1,genre] += rating
         
userProfile.head()


Movie id:  72505  rating:  8.0
Comedy
Romance
Movie id:  507882  rating:  6.0
Action
Adventure
Fantasy
Movie id:  198210  rating:  7.2
Drama
Fantasy
7.2


,Comedy,Romance,Action,Adventure,Fantasy,Drama
1,8.0,8.0,6.0,6.0,13.2,7.2


In [9]:
sumOfRatings = userProfile.iloc[0].sum()

for column in userProfile.columns:
    userProfile[column] = userProfile[column] / sumOfRatings
userProfile

,Comedy,Romance,Action,Adventure,Fantasy,Drama
1,0.165289,0.165289,0.123967,0.123967,0.272727,0.14876


In [10]:
dataSet["score"] = pd.Series(0, index = dataSet.index)
dataSet["score2"] = pd.Series(0, index= dataSet.index)
dataSet["score3"] = pd.Series(0, index = dataSet.index)
dataSet.head(5)

,tconst,averageRating,numVotes,titleType,primaryTitle,startYear,runtimeMinutes,genres,movieId,score,score2,score3
0,tt0000004,5.6,178,short,Un bon bock,1892,12,"Animation,Short",0,0,0,0
1,tt0000009,5.3,204,movie,Miss Jerry,1894,45,Romance,1,0,0,0
2,tt0000147,5.3,469,movie,The Corbett-Fitzsimmons Fight,1897,100,"Documentary,News,Sport",2,0,0,0
3,tt0000218,5.9,662,short,L'affaire Dreyfus,1899,13,"Biography,Drama,Short",3,0,0,0
4,tt0000335,6.1,55,short,Soldiers of the Cross,1900,40,"Biography,Drama,Short",4,0,0,0


In [11]:
from tqdm import tqdm
genres = set()
progress_bar = tqdm(total=len(dataSet))
for index,row in dataSet.iterrows():
    genres = row["genres"].split(",")
    score = 0
    score2 = 0
    for genre in genres:
        if genre in userProfile.columns:
            score += userProfile.iloc[0][genre] 
    dataSet.at[index, "score"] = score
    
    score2 = row["averageRating"] * score
    dataSet.at[index, "score2"] = score2
    
    score3 = row["numVotes"] * score2
    dataSet.at[index, "score3"] = score3
    progress_bar.update(1)
    
progress_bar.close()    
dataSet.head(5)    

100%|██████████| 862042/862042 [03:25<00:00, 4203.86it/s]


,tconst,averageRating,numVotes,titleType,primaryTitle,startYear,runtimeMinutes,genres,movieId,score,score2,score3
0,tt0000004,5.6,178,short,Un bon bock,1892,12,"Animation,Short",0,0.000000,0.000000,0.000000
1,tt0000009,5.3,204,movie,Miss Jerry,1894,45,Romance,1,0.165289,0.876033,178.710744
2,tt0000147,5.3,469,movie,The Corbett-Fitzsimmons Fight,1897,100,"Documentary,News,Sport",2,0.000000,0.000000,0.000000
3,tt0000218,5.9,662,short,L'affaire Dreyfus,1899,13,"Biography,Drama,Short",3,0.148760,0.877686,581.028099
4,tt0000335,6.1,55,short,Soldiers of the Cross,1900,40,"Biography,Drama,Short",4,0.148760,0.907438,49.909091


In [12]:
dataSet = dataSet.sort_values(by="score3", ascending=False)
dataSet.tail(5)

,tconst,averageRating,numVotes,titleType,primaryTitle,startYear,runtimeMinutes,genres,movieId,score,score2,score3
660383,tt3535126,7.3,11,tvEpisode,White Knuckles,2013,43,"Documentary,Reality-TV",660383,0.0,0.0,0.0
660384,tt3535138,7.0,11,tvEpisode,Grin and Bear It,2013,42,"Documentary,Reality-TV",660384,0.0,0.0,0.0
660385,tt3535156,7.8,19,tvEpisode,The Einstein Gamble,2014,43,"Documentary,Reality-TV",660385,0.0,0.0,0.0
660387,tt3535354,4.8,5,tvEpisode,Elimination 1,2014,57,"Game-Show,Reality-TV",660387,0.0,0.0,0.0
0,tt0000004,5.6,178,short,Un bon bock,1892,12,"Animation,Short",0,0.0,0.0,0.0
